# SPLADE v2 PT-BR - Treinamento Corrigido

Este notebook contém todas as correções necessárias para rodar o treinamento do SPLADE em 2025, resolvendo incompatibilidades de bibliotecas (AdamW/Hydra) e dependências de arquivos.

## Correções Aplicadas:
1. ✅ Caminhos corrigidos de `/content/` para caminhos relativos
2. ✅ Download de datasets com verificação de sucesso
3. ✅ Correção do dataloader para lidar com valores None
4. ✅ Dependências atualizadas (torch, transformers, etc.)

**Atenção:** Certifique-se de usar um Runtime com GPU (T4 ou A100) ou CPU se não houver GPU disponível.

In [7]:
# 1. Instalação de Bibliotecas
# !pip install pytrec_eval
# !pip install git+https://github.com/leobavila/splade.git -q
# !pip install hydra-core --upgrade

In [8]:
# 2. Clonagem e Patch de Correção (AdamW)
import os

# Clona o repositório se não existir
if not os.path.exists("splade"):
    os.system("git clone https://github.com/leobavila/splade.git")

# Corrige o erro de importação do AdamW (Transformers antigo vs novo)
file_path = "splade/splade/optim/bert_optim.py"
if os.path.exists(file_path):
    with open(file_path, "r") as f:
        content = f.read()

    new_content = content.replace(
        "from transformers.optimization import AdamW, get_linear_schedule_with_warmup",
        "from transformers import get_linear_schedule_with_warmup; from torch.optim import AdamW"
    )

    with open(file_path, "w") as f:
        f.write(new_content)
    print("✅ Patch aplicado: bert_optim.py corrigido.")
else:
    print("❌ Erro: Arquivo bert_optim.py não encontrado.")

✅ Patch aplicado: bert_optim.py corrigido.


In [9]:
# 3. Download e Preparação dos Datasets (mMARCO e mRobust)
import shutil
import os
from huggingface_hub import hf_hub_download

print("⏳ Baixando datasets públicos usando HuggingFace Hub... (Pode levar alguns minutos)")

# Criar pastas base (usando caminhos relativos ao projeto)
data_dir = "./data"
os.makedirs(f"{data_dir}/m_marco", exist_ok=True)
os.makedirs(f"{data_dir}/m_robust", exist_ok=True)

# Criar pastas de destino do SPLADE
os.makedirs("splade/data/pt/triplets", exist_ok=True)
os.makedirs("splade/data/pt/val_retrieval/collection", exist_ok=True)
os.makedirs("splade/data/pt/val_retrieval/queries", exist_ok=True)

def download_from_hf(repo_id, filename, output_path, description):
    """Download de arquivo do HuggingFace Hub"""
    if os.path.exists(output_path) and os.path.getsize(output_path) > 100:  # Mínimo 100 bytes
        print(f"✅ {description} já existe ({os.path.getsize(output_path)} bytes), pulando download.")
        return True
    
    print(f"📥 Baixando {description}...")
    try:
        downloaded_path = hf_hub_download(
            repo_id=repo_id,
            filename=filename,
            repo_type="dataset",
            local_dir=None
        )
        # Copiar para o destino desejado
        shutil.copy(downloaded_path, output_path)
        if os.path.exists(output_path) and os.path.getsize(output_path) > 100:
            print(f"✅ {description} baixado com sucesso ({os.path.getsize(output_path)} bytes)")
            return True
        else:
            print(f"❌ {description} falhou: arquivo muito pequeno ou vazio")
            return False
    except Exception as e:
        print(f"❌ Erro ao baixar {description}: {e}")
        return False

# --- mMARCO (Treino) ---
print("\n📦 Baixando datasets mMARCO...")
download_from_hf(
    "unicamp-dl/mmarco",
    "data/google/queries/train/portuguese_queries.train.tsv",
    f"{data_dir}/m_marco/queries_train.tsv",
    "queries_train.tsv"
)

download_from_hf(
    "unicamp-dl/mmarco",
    "data/google/collections/portuguese_collection.tsv",
    f"{data_dir}/m_marco/corpus.tsv",
    "corpus.tsv"
)

download_from_hf(
    "unicamp-dl/mmarco",
    "data/triples.train.ids.small.tsv",
    f"{data_dir}/m_marco/triples.train.ids.small.tsv",
    "triples.train.ids.small.tsv"
)

# Verificar se os arquivos foram baixados corretamente
mmarco_files = ["queries_train.tsv", "corpus.tsv", "triples.train.ids.small.tsv"]
if all(os.path.exists(f"{data_dir}/m_marco/{f}") and os.path.getsize(f"{data_dir}/m_marco/{f}") > 100 
       for f in mmarco_files):
    # Copiar para estrutura SPLADE
    shutil.copy(f"{data_dir}/m_marco/corpus.tsv", "splade/data/pt/triplets/corpus.tsv")
    shutil.copy(f"{data_dir}/m_marco/queries_train.tsv", "splade/data/pt/triplets/queries_train.tsv")
    shutil.copy(f"{data_dir}/m_marco/triples.train.ids.small.tsv", "splade/data/pt/triplets/raw.tsv")
    print("✅ Arquivos mMARCO copiados para estrutura SPLADE")
else:
    print("❌ Erro: Alguns arquivos mMARCO não foram baixados corretamente")
    print("Verificando arquivos:")
    for f in mmarco_files:
        path = f"{data_dir}/m_marco/{f}"
        if os.path.exists(path):
            print(f"  - {f}: {os.path.getsize(path)} bytes")
        else:
            print(f"  - {f}: NÃO ENCONTRADO")

# --- mRobust (Validação) ---
print("\n📦 Baixando datasets mRobust...")
download_from_hf(
    "unicamp-dl/mrobust",
    "data/queries/portuguese_queries.tsv",
    f"{data_dir}/m_robust/queries.tsv",
    "mrobust queries.tsv"
)

download_from_hf(
    "unicamp-dl/mrobust",
    "data/collections/portuguese_collection.tsv",
    f"{data_dir}/m_robust/corpus.tsv",
    "mrobust corpus.tsv"
)

download_from_hf(
    "unicamp-dl/mrobust",
    "qrels.robust04.txt",
    f"{data_dir}/m_robust/qrels.robust04.txt",
    "qrels.robust04.txt"
)

# Verificar e copiar arquivos mRobust
mrobust_files = ["queries.tsv", "corpus.tsv", "qrels.robust04.txt"]
if all(os.path.exists(f"{data_dir}/m_robust/{f}") and os.path.getsize(f"{data_dir}/m_robust/{f}") > 100 
       for f in mrobust_files):
    shutil.copy(f"{data_dir}/m_robust/corpus.tsv", "splade/data/pt/val_retrieval/collection/raw.tsv")
    shutil.copy(f"{data_dir}/m_robust/queries.tsv", "splade/data/pt/val_retrieval/queries/raw.tsv")
    print("✅ Arquivos mRobust copiados para estrutura SPLADE")
else:
    print("❌ Erro: Alguns arquivos mRobust não foram baixados corretamente")
    print("Verificando arquivos:")
    for f in mrobust_files:
        path = f"{data_dir}/m_robust/{f}"
        if os.path.exists(path):
            print(f"  - {f}: {os.path.getsize(path)} bytes")
        else:
            print(f"  - {f}: NÃO ENCONTRADO")

print("\n✅ Processo de download concluído.")

⏳ Baixando datasets públicos usando HuggingFace Hub... (Pode levar alguns minutos)

📦 Baixando datasets mMARCO...
✅ queries_train.tsv já existe (39281063 bytes), pulando download.
✅ corpus.tsv já existe (3431011785 bytes), pulando download.
✅ triples.train.ids.small.tsv já existe (905211990 bytes), pulando download.
✅ Arquivos mMARCO copiados para estrutura SPLADE

📦 Baixando datasets mRobust...
✅ mrobust queries.tsv já existe (28418 bytes), pulando download.
✅ mrobust corpus.tsv já existe (1914138316 bytes), pulando download.
✅ qrels.robust04.txt já existe (6543541 bytes), pulando download.
✅ Arquivos mRobust copiados para estrutura SPLADE

✅ Processo de download concluído.


In [10]:
# 4. Converter QRELS para JSON
import json
from collections import defaultdict
import os

qrel = defaultdict(dict)
data_dir = "./data"
qrel_path = f"{data_dir}/m_robust/qrels.robust04.txt"

if os.path.exists(qrel_path):
    with open(qrel_path, 'r') as file:
        for line in file:
            fields = line.split()
            if len(fields) >= 4:
                q_id = fields[0]
                doc_id = fields[2]
                rel = fields[3]
                qrel[q_id][doc_id] = int(rel)

    with open('splade/data/pt/val_retrieval/qrel.json', 'w') as file:
        json.dump(qrel, file)
    print("✅ QREL convertido para JSON.")
else:
    print(f"❌ Erro: qrels.robust04.txt não encontrado em {qrel_path}.")

✅ QREL convertido para JSON.


In [11]:
# 5. Gerar Arquivos de Configuração (CRÍTICO: Inclusão do parâmetro 'loss')
# Correção: Adicionado 'loss: InBatchPairwiseNLL' para corrigir o ConfigKeyError.

import os

# Criar estrutura de pastas
os.makedirs("splade/conf/train/config", exist_ok=True)
os.makedirs("splade/conf/train/data", exist_ok=True)
os.makedirs("splade/conf/train/model", exist_ok=True)
os.makedirs("splade/conf/index", exist_ok=True)
os.makedirs("splade/conf/retrieve_evaluate", exist_ok=True)
os.makedirs("splade/conf/flops", exist_ok=True)

# 5.1 Modelo
with open("splade/conf/train/model/splade_bertimbau_base.yaml", "w") as f:
    f.write("""
_target_: splade.models.transformer_rep.Splade
# Nota: O parâmetro real será lido do init_dict abaixo
model_type_or_dir: neuralmind/bert-base-portuguese-cased
    """)

# 5.2 Dados
with open("splade/conf/train/data/pt.yaml", "w") as f:
    f.write(f"""
# @package _global_
data:
    type: triplets
    TRAIN_DATA_DIR: {os.getcwd()}/splade/data/pt/triplets
    VALIDATION_DATA_DIR: {os.getcwd()}/splade/data/pt/val_retrieval
    QREL_PATH: {os.getcwd()}/splade/data/pt/val_retrieval/qrel.json
    """)

# 5.3 Config de Treino (CORREÇÃO AQUI: Adicionado 'loss')
with open("splade/conf/train/config/splade_pt.yaml", "w") as f:
    f.write("""
# @package _global_
config:
    lr: 2e-5
    seed: 123
    gradient_accumulation_steps: 1
    weight_decay: 0.01
    validation_metrics: [MRR@10]
    pretrained_no_yaml_config: false
    nb_iterations: 150000
    train_batch_size: 32
    eval_batch_size: 32
    index_retrieval_batch_size: 32
    record_frequency: 1000
    train_monitoring_freq: 500
    warmup_steps: 6000
    max_length: 256
    fp16: true
    matching_type: splade
    monitoring_ckpt: true
    tokenizer_type: neuralmind/bert-base-portuguese-cased

    # Parâmetro de perda que faltava:
    loss: InBatchPairwiseNLL

    # Chaves obrigatórias para o Hydra:
    checkpoint_dir: ""
    index_dir: ""
    out_dir: ""

    regularization:
        FLOPS:
            lambda_q: 0.0003
            lambda_d: 0.0001
            T: 50000
    """)

# 5.4 Config Geral
with open("splade/conf/config_splade_pt.yaml", "w") as f:
    f.write("""
defaults:
  - train/data: pt
  - train/model: splade_bertimbau_base
  - train/config: splade_pt
  - index: pt
  - retrieve_evaluate: pt
  - flops: pt
  - _self_

# init_dict com as correções anteriores
init_dict:
  model_type_or_dir: neuralmind/bert-base-portuguese-cased
  fp16: true

hydra:
  run:
    dir: experiments/pt/out
  job:
    chdir: true
    """)

# 5.5 Placeholders
with open("splade/conf/index/pt.yaml", "w") as f: f.write("# Placeholder")
with open("splade/conf/retrieve_evaluate/pt.yaml", "w") as f: f.write("# Placeholder")
with open("splade/conf/flops/pt.yaml", "w") as f: f.write("# Placeholder")

print("✅ Configurações recriadas com sucesso (loss: InBatchPairwiseNLL adicionado).")

✅ Configurações recriadas com sucesso (loss: InBatchPairwiseNLL adicionado).


In [ ]:
# 6. Executar Treinamento
import os
import sys
import subprocess

# Configurar ambiente
os.environ['PYTHONPATH'] = os.environ.get('PYTHONPATH', '') + ":" + os.path.join(os.getcwd(), 'splade')
os.environ['SPLADE_CONFIG_NAME'] = "config_splade_pt.yaml"
os.environ['PYTHONUNBUFFERED'] = '1'  # Desabilita buffering para ver logs em tempo real

print("🚀 Iniciando treinamento... Acompanhe os logs abaixo.")
print("Nota: Ignorar avisos 'Unable to register cuFFT/cuDNN' do TensorFlow/JAX.")
print("=" * 80)
print()

# Executar treinamento com output em tempo real usando subprocess
cmd = [
    sys.executable,  # Usa o Python do ambiente virtual
    '-m', 'splade.train_from_triplets_ids',
    'config.checkpoint_dir=experiments/pt/checkpoint',
    'config.index_dir=experiments/pt/index',
    'config.out_dir=experiments/pt/out'
]

# Mudar para o diretório splade
original_dir = os.getcwd()
os.chdir('splade')

try:
    # Executar com output em tempo real
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1,  # Line buffered
        env=os.environ.copy()
    )
    
    # Imprimir output em tempo real
    for line in process.stdout:
        print(line, end='', flush=True)
    
    process.wait()
    
    if process.returncode != 0:
        print(f"\n❌ Treinamento finalizado com código de saída: {process.returncode}")
    else:
        print("\n✅ Treinamento concluído com sucesso!")
        
except KeyboardInterrupt:
    print("\n⚠️ Treinamento interrompido pelo usuário.")
    if 'process' in locals():
        process.terminate()
except Exception as e:
    print(f"\n❌ Erro durante o treinamento: {e}")
    import traceback
    traceback.print_exc()
finally:
    os.chdir(original_dir)

🚀 Iniciando treinamento... Acompanhe os logs abaixo.
Nota: Ignorar avisos 'Unable to register cuFFT/cuDNN' do TensorFlow/JAX.

/home/user/Projects/SPLADE-PT-BR/splade/splade/train_from_triplets_ids.py:20: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=CONFIG_PATH, config_name=CONFIG_NAME)
data:
  type: triplets
  TRAIN_DATA_DIR: /home/user/Projects/SPLADE-PT-BR/splade/data/pt/triplets
  VALIDATION_DATA_DIR: /home/user/Projects/SPLADE-PT-BR/splade/data/pt/val_retrieval
  QREL_PATH: /home/user/Projects/SPLADE-PT-BR/splade/data/pt/val_retrieval/qrel.json
train:
  model:
    _target_: splade.models.transformer_rep.Splade
    model_type_or_dir: neuralmind/bert-base-portuguese-cased
config:
  lr: 2.0e-05
  seed: 123
  gradient_accumulation_steps: 4
  weight_decay: 0.01
  validation_metrics:
  - MRR@10
  pretrained_no_yaml_config: false
  nb_iterations: 150000
  tr